# Task 4: Portfolio Optimization Based on Forecasts
### This notebook will:

1. Combine historical and forecasted data for each asset (TSLA, BND, SPY).
2. Compute expected returns and volatility for each asset based on forecasted prices.
3. Optimize the portfolio allocation to maximize the Sharpe Ratio (risk-adjusted return).
4. Analyze the optimized portfolio’s risk, returns, and volatility, and visualize the cumulative performance.

### Import necessary libraries

In [1]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

### Define paths to data and models

In [2]:
# Define paths to data and models
os.chdir(r'c:\users\ermias.tadesse\10x\Predictive-Analysis-And-Modeling')
data_folder_path = 'Data/'
models_folder_path = 'Models/'

### Load Historical Data and Forecasts
We’ll load historical data and append forecasted data to calculate expected returns and volatility for each asset.

In [4]:
# Define tickers
tickers = ['TSLA', 'BND', 'SPY']

# Dictionary to store historical and forecasted data
data = {}

for ticker in tickers:
    # Load historical data
    processed_data_path = os.path.join(data_folder_path, 'processed', f'{ticker}_train.csv')
    historical_data = pd.read_csv(processed_data_path, parse_dates=['Date'], index_col='Date')
    
    # Load forecasted data generated in Task 3
    forecast_path = os.path.join(data_folder_path, 'processed', f'{ticker}_forecast.csv')
    forecast_data = pd.read_csv(forecast_path, parse_dates=['Date'], index_col='Date')
    
    # Concatenate historical and forecasted data
    combined_data = pd.concat([historical_data['Close'], forecast_data['Forecast']], axis=0)
    data[ticker] = combined_data

    # Display the last few rows of combined data
    print(f"\n{ticker} Data (Last 5 Rows):\n", combined_data.tail())



TSLA Data (Last 5 Rows):
 Date
2024-02-22 00:00:00+00:00    316.41122
2024-02-23 00:00:00+00:00    316.56805
2024-02-26 00:00:00+00:00    316.72305
2024-02-27 00:00:00+00:00    316.87634
2024-02-28 00:00:00+00:00    317.02798
dtype: float64

BND Data (Last 5 Rows):
 Date
2024-02-22 00:00:00+00:00    70.71255
2024-02-23 00:00:00+00:00    70.71100
2024-02-26 00:00:00+00:00    70.70946
2024-02-27 00:00:00+00:00    70.70793
2024-02-28 00:00:00+00:00    70.70642
dtype: float64

SPY Data (Last 5 Rows):
 Date
2024-02-22 00:00:00+00:00    419.24100
2024-02-23 00:00:00+00:00    419.27320
2024-02-26 00:00:00+00:00    419.30518
2024-02-27 00:00:00+00:00    419.33694
2024-02-28 00:00:00+00:00    419.36838
dtype: float64


### Calculate Daily Returns and Expected Annual Return
We’ll calculate daily returns based on forecasted prices and use them to compute the expected annual return for each asset.

In [5]:
# Calculate daily returns for each asset
returns = pd.DataFrame({ticker: data[ticker].pct_change().dropna() for ticker in tickers})

# Calculate expected annual return for each asset
annual_returns = returns.mean() * 252  # Assuming 252 trading days

# Display annual returns
print("Expected Annual Returns:\n", annual_returns)


Expected Annual Returns:
 TSLA    0.774170
BND    -0.038926
SPY     0.084380
dtype: float64


###  Calculate Covariance Matrix
The covariance matrix of returns will help us understand the co-movement of asset prices, which is crucial for portfolio risk management

In [6]:
# Calculate the covariance matrix for the asset returns
cov_matrix = returns.cov() * 252  # Annualized covariance

# Display the covariance matrix
print("Annualized Covariance Matrix:\n", cov_matrix)


Annualized Covariance Matrix:
           TSLA       BND       SPY
TSLA  0.399901  0.004166  0.069983
BND   0.004166  0.004259  0.003021
SPY   0.069983  0.003021  0.046543
